In [ ]:
# Copyright 2020 TU Ilmenau. All Rights Reserved.
#
# Code for the Particle detection baseline for the 
# On the use of a cascaded convolutional neural network for three-dimensional flow measurements using astigmatic PTV task.


In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import glob

from distutils.version import StrictVersion
from collections import defaultdict

from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0" 


In [2]:
# This is needed to display the images.
%matplotlib inline

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

In [4]:

MODEL_NAME = ''
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + ''

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'particle_classes_label_map.pbtxt')
NUM_CLASSES = 1

In [5]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [6]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [7]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [8]:
#path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = ''
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*.png'))
[TEST_IMAGE_PATHS.remove(x) for x in list(reversed(TEST_IMAGE_PATHS)) if x.find('_bb')!=-1]
# Size, in inches, of the output images.
IMAGE_SIZE = (2560, 2160)

In [9]:
import pandas as pd
TBL_FILENAME='FIlE_ID'
TBL_XMIN='xmin'
TBL_YMIN='ymin'
TBL_XMAX='xmax'
TBL_YMAX='ymax'
TBL_HEADER=[TBL_FILENAME,TBL_XMIN,TBL_YMIN,TBL_XMAX,TBL_YMAX]
CSV_BASEDIR = ''
df=pd.read_csv(os.path.join(CSV_BASEDIR, ''),sep=',', names=TBL_HEADER)


fname_list=[]
class_list=[]
xmin_list=[]
ymin_list=[]
xmax_list=[]
ymax_list=[]


In [ ]:
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    # Definite input and output Tensors for detection_graph
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    # Each box represents a part of the image where a particular object was detected.
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    for image_path in TEST_IMAGE_PATHS:
      image = Image.open(image_path)
      # the array based representation of the image will be used later in order to prepare the
      # result image with boxes and labels on it.
      image_np = load_image_into_numpy_array(image)
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      # Actual detection.
      (boxes, scores, classes, num) = sess.run(
          [detection_boxes, detection_scores, detection_classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
      # Visualization of the results of a detection.
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=8,
          min_score_thresh=0.1)
      image = Image.fromarray(image_np, 'RGB')
      image.save(image_path.replace('.png', '_bb3.png').replace('.jpeg', '_bb3.jpeg'))
     
      image.close()
      min_score_thresh = 0.1
      i=0
      pdclass=np.squeeze(classes).astype(np.int32)
      boxessq=np.squeeze(boxes)
      scoresq=np.squeeze(scores)
      bboxes = boxessq[scoresq > min_score_thresh]
      img_id = int(image_path.split('/')[-1].split('.')[0][-4:])
      print(img_id)
      final_box = []
      
      
      for box in bboxes:
           
        ymin, xmin, ymax, xmax = box
        final_box.append([xmin , xmax , ymin , ymax ])
        fname_list.append(img_id)
        class_list.append(pdclass[i])
        xmin_list.append(box[1]* 2560)
        ymin_list.append(box[0]*2160)
        xmax_list.append(box[3]*2560)
        ymax_list.append(box[2]*2160)
        i=i+1
      
    
    
    df_new = pd.DataFrame(data={
          'Filename':fname_list,
          'class':class_list,
          'xmin':xmin_list,
          'ymin':ymin_list,
          'xmax':xmax_list,
          'ymax':ymax_list, 
      })
new_csv = os.path.join(CSV_BASEDIR, '')
df_new.to_csv(new_csv, sep=',',header=False, index=False)  